# Finding interval Estimators

Let X be a continuous Gaussian random variable with CDF F_X (x) = P(X\leq x).
Suppose that we are interested in finding two values x_h and x_l such that
P(x_l \leq X x_h) \leq 1-\alpha

In [ ]:
x_l = F^-1_X